In [ ]:
import pandas as pd
import scipy.stats as st
import numpy as np
from sklearn import ensemble
import sklearn as skl
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
import math
from sklearn.model_selection import train_test_split
%pylab inline

In [ ]:
data=pd.read_csv('Dataset.csv',header=None)
Y=data.loc[:,data.columns==43]
data=np.array(data)
Y=np.array(Y)
Y=Y.reshape(Y.shape[0])
tmpY=np.zeros(Y.shape[0])
tmpY[(Y>0.2)&(Y<25)]=1.
tmpY[Y<0.2]=2.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( data, tmpY, test_size=0.40, random_state=42)

In [ ]:
scaler=skl.preprocessing.FunctionTransformer(func=lambda x: 1 / (1 + np.exp(-x)))
scaler=scaler.fit(data[:,:43])

X_t=X_train[:,:43]
Y_t=y_train
tmpX=scaler.transform(X_t)
# modelClassify=skl.svm.SVC(C=70,kernel='poly',probability=True,random_state=42,class_weight='balanced')
# modelClassify=skl.linear_model.LogisticRegression(C=10,random_state=42,class_weight='balanced')
# modelClassify=skl.ensemble.RandomForestClassifier(n_estimators=10,criterion='entropy', class_weight='balanced', random_state=42)
# modelClassify=skl.tree.DecisionTreeClassifier(criterion='gini', class_weight='balanced', random_state=42)
modelClassify=skl.neighbors.KNeighborsClassifier(n_neighbors=2,weights='distance')
modelClassify.fit(tmpX,Y_t)
Xte=X_test[:,:43]
Yte=y_test
tmpX=scaler.transform(Xte)
y_p = modelClassify.predict(tmpX)
cm = skl.metrics.confusion_matrix(Yte, y_p)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print(skl.metrics.precision_score(Yte,y_p,average='macro'))
print(skl.metrics.recall_score(Yte,y_p,average='macro'))
print(skl.metrics.f1_score(Yte,y_p,average='macro'))
print(skl.metrics.accuracy_score(Yte,y_p))
print(cm.diagonal())


In [ ]:
kf = skl.model_selection.StratifiedKFold(n_splits=5,shuffle=True,random_state=42)

In [ ]:
scaler=skl.preprocessing.FunctionTransformer(func=lambda x: 1 / (1 + np.exp(-x)))
scaler=scaler.fit(data[:,:43])

cv_score={'pre':0,'rec':0,'f1_score':0,'acc':0}
cv_score2={'pre':np.array([0.,0.,0.]),'rec':np.array([0.,0.,0.]),'f1_score':np.array([0.,0.,0.]),'acc':np.array([0.,0.,0.])}
for train_index, test_index in kf.split(data,tmpY):
    X_t=data[train_index,:43]
    Y_t=tmpY[train_index]
    tmpX=scaler.transform(X_t)
    modelClassify=skl.tree.DecisionTreeClassifier(criterion='gini', class_weight='balanced', random_state=42)
    modelClassify.fit(tmpX,Y_t)
    Xte=data[test_index,:43]
    Yte=tmpY[test_index]
    tmpX=scaler.transform(Xte)
    y_p = modelClassify.predict(tmpX)
    cv_score['pre']+=skl.metrics.precision_score(Yte,y_p,average='macro')
    cv_score['rec']+=skl.metrics.recall_score(Yte,y_p,average='macro')
    cv_score['f1_score']+=skl.metrics.f1_score(Yte,y_p,average='macro')
    cv_score['acc']+=skl.metrics.accuracy_score(Yte,y_p)
    cm = skl.metrics.confusion_matrix(Yte, y_p)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    cv_score2['acc']+=np.array(cm.diagonal())
    tt=skl.metrics.precision_recall_fscore_support(Yte,y_p)
    cv_score2['pre']+=tt[0]
    cv_score2['rec']+=tt[1]
    cv_score2['f1_score']+=tt[2]
cv_score,cv_score2

In [ ]:
print(cv_score['pre']/5,cv_score['rec']/5,cv_score['acc']/5,cv_score['f1_score']/5)
print(cv_score2['pre']/5,cv_score2['rec']/5,cv_score2['acc']/5,cv_score2['f1_score']/5)

In [ ]:
correlate=[]
for i in range(43):
    t1,t2=st.spearmanr(data[:,i],Y)
    if t2<0.0012:
        correlate.append((i,round(t1,4),round(t2,4)))
# [i[0] for i in correlate]

In [ ]:
tmpData=data[:,[i[0] for i in correlate]]

In [ ]:
scaler=skl.preprocessing.FunctionTransformer(func=lambda x: 1 / (1 + np.exp(-x)))
scaler=scaler.fit(data[:,:43])

cv_score={'pre':0,'rec':0,'f1_score':0,'acc':0}
cv_score2={'pre':np.array([0.,0.,0.]),'rec':np.array([0.,0.,0.]),'f1_score':np.array([0.,0.,0.]),'acc':np.array([0.,0.,0.])}
for train_index, test_index in kf.split(tmpData,tmpY):
    X_t=tmpData[train_index]
    Y_t=tmpY[train_index]
    tmpX=scaler.transform(X_t)
    modelClassify=skl.tree.DecisionTreeClassifier(criterion='gini', class_weight='balanced', random_state=42)
    modelClassify.fit(tmpX,Y_t)
    Xte=tmpData[test_index]
    Yte=tmpY[test_index]
    tmpX=scaler.transform(Xte)
    y_p = modelClassify.predict(tmpX)
    cv_score['pre']+=skl.metrics.precision_score(Yte,y_p,average='macro')
    cv_score['rec']+=skl.metrics.recall_score(Yte,y_p,average='macro')
    cv_score['f1_score']+=skl.metrics.f1_score(Yte,y_p,average='macro')
    cv_score['acc']+=skl.metrics.accuracy_score(Yte,y_p)
    cm = skl.metrics.confusion_matrix(Yte, y_p)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    cv_score2['acc']+=np.array(cm.diagonal())
    tt=skl.metrics.precision_recall_fscore_support(Yte,y_p)
    cv_score2['pre']+=tt[0]
    cv_score2['rec']+=tt[1]
    cv_score2['f1_score']+=tt[2]
print(cv_score,cv_score2)
print(cv_score['pre']/5,cv_score['rec']/5,cv_score['acc']/5,cv_score['f1_score']/5)
print(cv_score2['pre']/5,cv_score2['rec']/5,cv_score2['acc']/5,cv_score2['f1_score']/5)

In [ ]:
def RRMSE(A,Y):
  tmp = Y-A
  tmp = tmp*tmp*1.0
  A = A*A*1.0
  return np.sqrt(np.mean(tmp/A))
  # return (np.sqrt(np.sum(tmp)) / np.sqrt(np.sum(A))) / len(A)
def RMAE(A,Y):
  A = A * 1.0
  Y = Y * 1.0
  Y[A==0] += 0.1
  A[A==0] += 0.1
  tmp = Y / A
  tmp = np.abs(1 - tmp)
  return np.mean(tmp)


In [ ]:
scaler=skl.preprocessing.FunctionTransformer(func=lambda x: 1 / (1 + np.exp(-x)))
scaler=scaler.fit(data[:,:43])
poly=PolynomialFeatures(degree=2)
poly=poly.fit(data[:,:43])

X_t=X_train[:,:43]
Y_t=X_train[:,-1]
tmp_t=y_train
Xte=X_test[:,:43]
Yte=X_test[:,-1]
tmp_te=y_test
for c in [0,1,2]:
    X_t_=X_t[tmp_t==c]
    Y_t_=Y_t[tmp_t==c]
#     tmpX=poly.transform(scaler.transform(X_t_))
    tmpX=scaler.transform(X_t_)
    # tmpModel=skl.linear_model.LinearRegression(normalize=True)
#     tmpModel=skl.linear_model.Lasso(normalize=True,alpha=3,random_state=42)
    tmpModel=skl.ensemble.RandomForestRegressor(n_estimators=12,min_samples_split=2,random_state=42)
    # tmpModel=skl.tree.DecisionTreeRegressor(random_state=42)
    tmpModel.fit(tmpX,Y_t_)
    Xte_=Xte[tmp_te==c]
    Yte_=Yte[tmp_te==c]
#     tmpX=poly.transform(scaler.transform(Xte_))
    tmpX=scaler.transform(Xte_)
    tmpY2=tmpModel.predict(tmpX)
    print(c,mean_squared_error(Yte_,tmpY2),skl.metrics.mean_absolute_error(Yte_,tmpY2),RRMSE(Yte_,tmpY2),RMAE(Yte_,tmpY2))
# tmpX=poly.transform(scaler.transform(X_t))
tmpX=scaler.transform(X_t)
# tmpModel=skl.linear_model.LinearRegression(normalize=True)
# tmpModel=skl.linear_model.Lasso(normalize=True,alpha=3,random_state=42)
tmpModel=skl.ensemble.RandomForestRegressor(n_estimators=12,min_samples_split=2,random_state=42)
# tmpModel=skl.tree.DecisionTreeRegressor(random_state=42)
tmpModel.fit(tmpX,Y_t)
# tmpX=poly.transform(scaler.transform(Xte_))
tmpX=scaler.transform(Xte)
tmpY2=tmpModel.predict(tmpX)
print(mean_squared_error(Yte,tmpY2),skl.metrics.mean_absolute_error(Yte,tmpY2),RRMSE(Yte,tmpY2),RMAE(Yte,tmpY2))


In [ ]:
scaler=skl.preprocessing.FunctionTransformer(func=lambda x: 1 / (1 + np.exp(-x)))
scaler=scaler.fit(data[:,:43])

f_c=0
cv_score={'c1':np.array([0.,0.]),'c2':np.array([0.,0.]),'c3':np.array([0.,0.]),'all':np.array([0.,0.])}
for train_index, test_index in kf.split(data,tmpY):
    f_c+=1
    X_t=data[train_index,:43]
    Y_t=Y[train_index]
    tmp_t=tmpY[train_index]
    Xte=data[test_index,:43]
    Yte=Y[test_index]
    tmp_te=tmpY[test_index]
    for c in [0,1,2]:
        X_t_=X_t[tmp_t==c]
        Y_t_=Y_t[tmp_t==c]
        tmpX=scaler.transform(X_t_)
        tmpModel=skl.tree.DecisionTreeRegressor(random_state=42)
        tmpModel.fit(tmpX,Y_t_)
        Xte_=Xte[tmp_te==c]
        Yte_=Yte[tmp_te==c]
        tmpX=scaler.transform(Xte_)
        tmpY2=tmpModel.predict(tmpX)
        print(f_c,c,mean_squared_error(Yte_,tmpY2),skl.metrics.mean_absolute_error(Yte_,tmpY2))
        cv_score['c'+str(c+1)]+=[mean_squared_error(Yte_,tmpY2),skl.metrics.mean_absolute_error(Yte_,tmpY2)]
    tmpX=scaler.transform(X_t)
    tmpModel=skl.tree.DecisionTreeRegressor(random_state=42)
    tmpModel.fit(tmpX,Y_t)
    tmpX=scaler.transform(Xte)
    tmpY2=tmpModel.predict(tmpX)
    print(f_c,mean_squared_error(Yte,tmpY2),skl.metrics.mean_absolute_error(Yte,tmpY2))
    cv_score['all']+=[mean_squared_error(Yte,tmpY2),skl.metrics.mean_absolute_error(Yte,tmpY2)]
print(cv_score['c1']/5,cv_score['c2']/5,cv_score['c3']/5,cv_score['all']/5)

In [ ]:
scaler=skl.preprocessing.FunctionTransformer(func=lambda x: 1 / (1 + np.exp(-x)))
scaler=scaler.fit(tmpData)

f_c=0
cv_score={'c1':np.array([0.,0.]),'c2':np.array([0.,0.]),'c3':np.array([0.,0.]),'all':np.array([0.,0.])}
for train_index, test_index in kf.split(tmpData,tmpY):
    f_c+=1
    X_t=tmpData[train_index]
    Y_t=Y[train_index]
    tmp_t=tmpY[train_index]
    Xte=tmpData[test_index]
    Yte=Y[test_index]
    tmp_te=tmpY[test_index]
    for c in [0,1,2]:
        X_t_=X_t[tmp_t==c]
        Y_t_=Y_t[tmp_t==c]
        tmpX=scaler.transform(X_t_)
        tmpModel=skl.tree.DecisionTreeRegressor(random_state=42)
        tmpModel.fit(tmpX,Y_t_)
        Xte_=Xte[tmp_te==c]
        Yte_=Yte[tmp_te==c]
        tmpX=scaler.transform(Xte_)
        tmpY2=tmpModel.predict(tmpX)
        print(f_c,c,mean_squared_error(Yte_,tmpY2),skl.metrics.mean_absolute_error(Yte_,tmpY2))
        cv_score['c'+str(c+1)]+=[mean_squared_error(Yte_,tmpY2),skl.metrics.mean_absolute_error(Yte_,tmpY2)]
    tmpX=scaler.transform(X_t)
    tmpModel=skl.tree.DecisionTreeRegressor(random_state=42)
    tmpModel.fit(tmpX,Y_t)
    tmpX=scaler.transform(Xte)
    tmpY2=tmpModel.predict(tmpX)
    print(f_c,mean_squared_error(Yte,tmpY2),skl.metrics.mean_absolute_error(Yte,tmpY2))
    cv_score['all']+=[mean_squared_error(Yte,tmpY2),skl.metrics.mean_absolute_error(Yte,tmpY2)]
print(cv_score['c1']/5,cv_score['c2']/5,cv_score['c3']/5,cv_score['all']/5)